In [1]:
import torch.nn as nn
from torch.nn import functional as F

In [2]:
import torch

def check_gpu():
    if torch.cuda.is_available():
        device = 'cuda'
        print("GPUs are available.")
        print(f"Number of GPUs: {torch.cuda.device_count()}")
        print(f"Device is set to: {device}")
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    else:
        print("No GPUs available.")
        device = 'cpu'
        print(f"Device is set to: {device}")
    return device

device = check_gpu()


GPUs are available.
Number of GPUs: 1
Device is set to: cuda
GPU 0: NVIDIA GeForce RTX 4070


In [4]:
#M

In [3]:
block_size = 8 
batch_size = 4


In [4]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text= f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [5]:
string_to_int = { ch:i for i ,ch in enumerate(chars) }
int_to_string = { i:ch for i ,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [6]:
print(decode(encode('hello')))

hello


In [7]:
data = torch.tensor(encode(text), dtype = torch.long)

In [8]:
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,
         1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25,
        38, 28,  1, 39, 30,  1, 39, 50,  9,  1])


In [9]:
n = int(0.8 * len(data))
train_data = data[:n] 
val_data = data[n:] 


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x, y    

In [27]:
block_size = 8 

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = x[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(80)
when input is tensor([80, 28]) target is tensor(28)
when input is tensor([80, 28, 39]) target is tensor(39)
when input is tensor([80, 28, 39, 42]) target is tensor(42)
when input is tensor([80, 28, 39, 42, 39]) target is tensor(39)
when input is tensor([80, 28, 39, 42, 39, 44]) target is tensor(44)
when input is tensor([80, 28, 39, 42, 39, 44, 32]) target is tensor(32)
when input is tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is tensor(49)


In [10]:
x, y = get_batch('train')
x,y = x.to(device) , y.to(device)
print(f"The shape of x is: {x.shape}")
print(f"x is: {x}",  end='\n\n')
print(f"The shape of y is: {y.shape}")
print(f"y is: {y}")

tensor([177935,  54757,  60161, 148944])
The shape of x is: torch.Size([4, 8])
x is: tensor([[58, 67, 72, 62, 68, 67, 72,  1],
        [58, 72, 73, 71, 68, 78, 58, 57],
        [67, 58,  1, 54, 67, 57,  1, 69],
        [58, 72,  1, 76, 61, 62, 56, 61]], device='cuda:0')

The shape of y is: torch.Size([4, 8])
y is: tensor([[67, 72, 62, 68, 67, 72,  1, 74],
        [72, 73, 71, 68, 78, 58, 57,  1],
        [58,  1, 54, 67, 57,  1, 69, 58],
        [72,  1, 76, 61, 62, 56, 61,  1]], device='cuda:0')


In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super.__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self, index, targets):
        logits = self.token_embedding_table(index)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, index, max_new_tokens):

        for _ in range
